In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [15]:
df = pd.read_excel('../Data/legendary_lord_defeat_traits.xlsx')

positive_keywords = ['Missile resistance:', ' Armour-piercing weapon damage:', 'factionwide', 'Physical resistance:', 'Melee attack:', 'Armour:']
negative_keywords = ['when', 'Recruitment cost:', 'Spell resistance:', 'reef', 'storm', 'Hero action cost:', 'Hero self-defence:', 'Income from raiding:', 'Magic item drop chance:', 'Recruit rank:', 'Diplomatic relations:', 'Hero action success chance:']

def classify_effect(effect):
    effect = effect.lower()
    if any(word in effect for word in positive_keywords):
        return 1  # Good trait
    elif any(word in effect for word in negative_keywords):
        return 0  # Bad trait
    else:
        return -1  # Neutral or unknown

In [18]:
# Apply the classification function
df['Label'] = df['Effect'].apply(classify_effect)

# Filter out neutral or unknown labels
df = df[df['Label'] != -1]

In [19]:
df

,Trait,Lord that must be defeated,Effect,Description,Label
1,Crowreaver,Malagor the Dark Omen,Leadership: +10 when fighting against Beastmen...,The despoiler becomes the despoiled.,0
6,Hammer and Anvil,Alberic de Bordeleaux,Melee attack: +15 when fighting at sea,Outstanding personal courage and martial skill...,0
7,End of an Errant,Repanse de Lyonesse,Growth: +5 (local province) Double experience...,Bretonnia's blessed cleanser has herself been ...,1
10,De-Towered,Drazhoath the Ashen,Income from Settlement buildings: +10% (factio...,Having been drawn from the relative safety of ...,1
14,Black Ark Down,Lokhir Fellheart,Immune to storm and reef attrition ( Lord's army),"""You'd better make sure you kill him, or this ...",0
20,Beardhammer,Grombrindal - The White Dwarf,Attribute: Causes Terror when fighting against...,"This time, the tables have been turned against...",0
22,Grudgekiller,Thorgrim Grudgebearer,Construction cost: -10% for all buildings (Loc...,What to use to pad my mattress? I know - fines...,1
31,Weightwatcher,Grom the Paunch,Diplomatic relations: +10 with High Elves Att...,The bigger they are...,0
34,Ruin Unrestrained,Teclis,Winds of Magicpower reserve change: +10% when ...,"With Teclis defeated, who will drive back the ...",0
36,Neverqueen,Alarielle the Radiant,Control +4 (local province) Chance of Winds o...,The High Elves will have to find a new title f...,0
